<a href="https://colab.research.google.com/github/Didier06/IA_FABLAB/blob/main/Yolov8_Minst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ======================================================
# 🧠 MNIST multi-chiffres avec YOLOv8m (Ultralytics)
# ======================================================

!pip install ultralytics torchvision tqdm pyyaml --quiet

from torchvision import datasets
from PIL import Image
import random, os, yaml
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.8 MB/s eta 0:00:00


In [2]:
# ======================================================
# 1️⃣ Monter Google Drive pour sauvegarder le modèle
# ======================================================
from google.colab import drive
drive.mount('/content/drive')

save_dir = "/content/drive/MyDrive/MNIST_YOLOv8m"
os.makedirs(save_dir, exist_ok=True)
print(f"📁 Modèle sera sauvegardé dans : {save_dir}")

Mounted at /content/drive
📁 Modèle sera sauvegardé dans : /content/drive/MyDrive/MNIST_YOLOv8m


In [3]:
# ======================================================
# 2️⃣ Génération du dataset multi-chiffres (3 à 10)
# ======================================================
mnist = datasets.MNIST(root="data", train=True, download=True)

os.makedirs("mnist_yolo/images/train", exist_ok=True)
os.makedirs("mnist_yolo/labels/train", exist_ok=True)
os.makedirs("mnist_yolo/images/val", exist_ok=True)
os.makedirs("mnist_yolo/labels/val", exist_ok=True)

def create_images(nb_images, split):
    for i in tqdm(range(nb_images), desc=f"Génération {split}"):
        img = Image.new("L", (320, 320), color=0)
        nb_digits = random.randint(3, 10)
        label_lines = []

        for _ in range(nb_digits):
            idx = random.randint(0, len(mnist) - 1)
            digit_img, label = mnist[idx]
            x = random.randint(0, 320 - 28)
            y = random.randint(0, 320 - 28)
            img.paste(digit_img, (x, y))

            xc = (x + 14) / 320
            yc = (y + 14) / 320
            w = 28 / 320
            h = 28 / 320
            label_lines.append(f"{label} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

        img.save(f"mnist_yolo/images/{split}/{i}.jpg")
        with open(f"mnist_yolo/labels/{split}/{i}.txt", "w") as f:
            f.write("\n".join(label_lines))

create_images(4000, "train")
create_images(1000, "val")

100%|██████████| 9.91M/9.91M [00:01<00:00, 6.11MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 161kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.93MB/s]
Génération val: 100%|██████████| 1000/1000 [00:00<00:00, 1592.60it/s]


In [4]:
# ======================================================
# 3️⃣ Fichier de configuration data.yaml
# ======================================================
import yaml

with open("mnist_yolo/data.yaml", "w") as f:
    yaml.dump({
        'path': '/content/mnist_yolo',  # dossier racine du dataset
        'train': 'images/train',
        'val': 'images/val',
        'nc': 10,
        'names': [str(i) for i in range(10)]
    }, f)

print("✅ Dataset YOLOv8 prêt")


✅ Dataset YOLOv8 prêt


In [5]:
!nvidia-smi

Tue Oct 21 08:51:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# ======================================================
# Entraînement YOLOv8m — version 100% continue et robuste (avec reprise native)
# ======================================================
import os
from ultralytics import YOLO
from google.colab import drive

# 1️⃣ Monter Google Drive
drive.mount('/content/drive')

# 2️⃣ Répertoires
drive_dir = '/content/drive/MyDrive/yolo_mnist_training'
run_name = 'mnist_yolo_run'
data_yaml = 'mnist_yolo/data.yaml'
base_model = 'yolov8m.pt'

# 3️⃣ Vérifier s’il existe déjà un entraînement
run_dir = f"{drive_dir}/{run_name}"
last_ckpt = os.path.join(run_dir, 'weights', 'last.pt')

# 4️⃣ Charger le modèle
if os.path.exists(last_ckpt):
    print(f"🔁 Reprise automatique depuis {last_ckpt}")
    model = YOLO(last_ckpt)
    resume_mode = True
else:
    print("🆕 Nouveau départ avec yolov8m.pt")
    model = YOLO(base_model)
    resume_mode = False

# 5️⃣ Lancer un seul entraînement géré par YOLO
results = model.train(
    data=data_yaml,
    epochs=50,
    imgsz=320,
    project=drive_dir,   # les fichiers seront directement dans ton Drive
    name=run_name,
    resume=resume_mode,  # ✅ vraie reprise native
    save_period=5,       # ✅ sauvegarde tous les 5 epochs
    device=0
)

print("✅ Entraînement complet avec sauvegarde automatique sur Drive !")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🆕 Nouveau départ avec yolov8m.pt
Ultralytics 8.3.218 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=mnist_yolo/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale

In [ ]:
# ======================================================
# 5️⃣ Visualisation des courbes (loss et mAP)
# ======================================================
import pandas as pd
import matplotlib.pyplot as plt
metrics_path = "mnist_yolo_train/yolov8m/results.csv"
df = pd.read_csv(metrics_path)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(df['epoch'], df['train/box_loss'], label='Box Loss')
plt.plot(df['epoch'], df['train/cls_loss'], label='Class Loss')
plt.plot(df['epoch'], df['train/dfl_loss'], label='DFL Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Courbes de perte (Loss)')
plt.legend()

plt.subplot(1,2,2)
plt.plot(df['epoch'], df['metrics/mAP50'], label='mAP@50')
plt.plot(df['epoch'], df['metrics/mAP50-95'], label='mAP@50-95')
plt.xlabel('Epoch')
plt.ylabel('mAP')
plt.title('Précision moyenne (mAP)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# ======================================================
# 6️⃣ Test de prédiction
# ======================================================
results = model.predict("mnist_yolo/images/val/5.jpg", save=True, imgsz=320)
results[0].show()

# ======================================================
# 7️⃣ Sauvegarde du modèle entraîné dans Drive
# ======================================================
import shutil
src = "mnist_yolo_train/yolov8m/weights/best.pt"
dst = os.path.join(save_dir, "best.pt")
shutil.copy(src, dst)

print(f"✅ Modèle sauvegardé ici : {dst}")